# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv("cal_housing_clean.csv")

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_data = housing.drop("medianHouseValue", axis=1)

In [7]:
labels = housing["medianHouseValue"]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x_data, labels, test_size=0.3, random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()

In [11]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train = pd.DataFrame(data=scaler.transform(X_train), columns = X_train.columns, index=X_train.index)

In [13]:
X_test = pd.DataFrame(data=scaler.transform(X_test), columns = X_test.columns, index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [14]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [15]:
import tensorflow as tf

In [16]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [17]:
feat_cols = [age, rooms, bedrooms, pop, households, income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [18]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10, num_epochs=1000, shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [19]:
model = tf.estimator.DNNRegressor(hidden_units=[10, 10, 10], feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/jf/g8tp5ct56hl6p6r66zl2knzc0000gn/T/tmptj6toi03', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a2a9c6898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [20]:
model.train(input_fn=input_func, steps=100000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/jf/g8tp5ct56hl6p6r66zl2knzc0000gn/T/tmptj6toi03/model.ckpt.
INFO:tensorflow:loss = 358905380000.0, step = 1
INFO:tensorflow:global_step/sec: 545.818
INFO:tensorflow:loss = 394370100000.0, step = 101 (0.186 sec)
INFO:tensorflow:global_step/sec: 723.5
INFO:tensorflow:loss = 557725400000.0, step = 201 (0.137 sec)
INFO:tensorflow:global_step/sec: 402.891
INFO:tensorflow:loss = 730535500000.0, step = 301 (0.248 sec)
INFO:tensorflow:global_step/sec: 665.2
INFO:tensorflow:loss = 438984770000.0, step = 401 (0.149 sec)
INFO:tensorflow:global_step/sec: 707.709
INFO:tensorflow:loss = 304802100000.0, step = 501 (0.140 sec)
INFO:tensorflow:global_step/sec: 687.601
INFO:tensorflow:loss = 304307500000.0, step 

INFO:tensorflow:global_step/sec: 749.036
INFO:tensorflow:loss = 73886990000.0, step = 7701 (0.134 sec)
INFO:tensorflow:global_step/sec: 722.929
INFO:tensorflow:loss = 119469470000.0, step = 7801 (0.139 sec)
INFO:tensorflow:global_step/sec: 717.587
INFO:tensorflow:loss = 82676000000.0, step = 7901 (0.140 sec)
INFO:tensorflow:global_step/sec: 693.092
INFO:tensorflow:loss = 106138550000.0, step = 8001 (0.146 sec)
INFO:tensorflow:global_step/sec: 705.526
INFO:tensorflow:loss = 72179655000.0, step = 8101 (0.139 sec)
INFO:tensorflow:global_step/sec: 742.088
INFO:tensorflow:loss = 60928004000.0, step = 8201 (0.136 sec)
INFO:tensorflow:global_step/sec: 774.504
INFO:tensorflow:loss = 103492690000.0, step = 8301 (0.130 sec)
INFO:tensorflow:global_step/sec: 787.58
INFO:tensorflow:loss = 172645140000.0, step = 8401 (0.127 sec)
INFO:tensorflow:global_step/sec: 737.578
INFO:tensorflow:loss = 71229006000.0, step = 8501 (0.135 sec)
INFO:tensorflow:global_step/sec: 670.101
INFO:tensorflow:loss = 123472

INFO:tensorflow:global_step/sec: 613.271
INFO:tensorflow:loss = 92157970000.0, step = 15601 (0.163 sec)
INFO:tensorflow:global_step/sec: 638.035
INFO:tensorflow:loss = 21935358000.0, step = 15701 (0.157 sec)
INFO:tensorflow:global_step/sec: 595.586
INFO:tensorflow:loss = 63008715000.0, step = 15801 (0.168 sec)
INFO:tensorflow:global_step/sec: 738.264
INFO:tensorflow:loss = 68216180000.0, step = 15901 (0.137 sec)
INFO:tensorflow:global_step/sec: 732.477
INFO:tensorflow:loss = 89816610000.0, step = 16001 (0.137 sec)
INFO:tensorflow:global_step/sec: 672.346
INFO:tensorflow:loss = 24845752000.0, step = 16101 (0.145 sec)
INFO:tensorflow:global_step/sec: 693.236
INFO:tensorflow:loss = 29659464000.0, step = 16201 (0.144 sec)
INFO:tensorflow:global_step/sec: 735.576
INFO:tensorflow:loss = 73563670000.0, step = 16301 (0.138 sec)
INFO:tensorflow:global_step/sec: 719.026
INFO:tensorflow:loss = 147353420000.0, step = 16401 (0.141 sec)
INFO:tensorflow:global_step/sec: 709.1
INFO:tensorflow:loss = 5

INFO:tensorflow:global_step/sec: 647.203
INFO:tensorflow:loss = 53877506000.0, step = 23501 (0.156 sec)
INFO:tensorflow:global_step/sec: 663.354
INFO:tensorflow:loss = 175050980000.0, step = 23601 (0.153 sec)
INFO:tensorflow:global_step/sec: 660.514
INFO:tensorflow:loss = 298607580000.0, step = 23701 (0.147 sec)
INFO:tensorflow:global_step/sec: 692.882
INFO:tensorflow:loss = 79497670000.0, step = 23801 (0.144 sec)
INFO:tensorflow:global_step/sec: 615.57
INFO:tensorflow:loss = 59834483000.0, step = 23901 (0.162 sec)
INFO:tensorflow:global_step/sec: 684.107
INFO:tensorflow:loss = 63395647000.0, step = 24001 (0.146 sec)
INFO:tensorflow:global_step/sec: 709.5
INFO:tensorflow:loss = 99078636000.0, step = 24101 (0.142 sec)
INFO:tensorflow:global_step/sec: 657.173
INFO:tensorflow:loss = 134208240000.0, step = 24201 (0.151 sec)
INFO:tensorflow:global_step/sec: 698.216
INFO:tensorflow:loss = 58717650000.0, step = 24301 (0.143 sec)
INFO:tensorflow:global_step/sec: 667.513
INFO:tensorflow:loss = 

INFO:tensorflow:global_step/sec: 746.263
INFO:tensorflow:loss = 118449310000.0, step = 31401 (0.133 sec)
INFO:tensorflow:global_step/sec: 763.196
INFO:tensorflow:loss = 78095520000.0, step = 31501 (0.132 sec)
INFO:tensorflow:global_step/sec: 691.677
INFO:tensorflow:loss = 87901760000.0, step = 31601 (0.143 sec)
INFO:tensorflow:global_step/sec: 697.545
INFO:tensorflow:loss = 226481440000.0, step = 31701 (0.143 sec)
INFO:tensorflow:global_step/sec: 739.099
INFO:tensorflow:loss = 43720147000.0, step = 31801 (0.138 sec)
INFO:tensorflow:global_step/sec: 725.174
INFO:tensorflow:loss = 60549497000.0, step = 31901 (0.137 sec)
INFO:tensorflow:global_step/sec: 740.279
INFO:tensorflow:loss = 46287065000.0, step = 32001 (0.134 sec)
INFO:tensorflow:global_step/sec: 734.539
INFO:tensorflow:loss = 60555428000.0, step = 32101 (0.137 sec)
INFO:tensorflow:global_step/sec: 740.181
INFO:tensorflow:loss = 21838758000.0, step = 32201 (0.134 sec)
INFO:tensorflow:global_step/sec: 757.174
INFO:tensorflow:loss 

...

INFO:tensorflow:global_step/sec: 618.877
INFO:tensorflow:loss = 81991040000.0, step = 63001 (0.162 sec)
INFO:tensorflow:global_step/sec: 639.959
INFO:tensorflow:loss = 51928480000.0, step = 63101 (0.155 sec)
INFO:tensorflow:global_step/sec: 694.541
INFO:tensorflow:loss = 22696850000.0, step = 63201 (0.144 sec)
INFO:tensorflow:global_step/sec: 684.806
INFO:tensorflow:loss = 31838372000.0, step = 63301 (0.146 sec)
INFO:tensorflow:global_step/sec: 684.288
INFO:tensorflow:loss = 70063560000.0, step = 63401 (0.147 sec)
INFO:tensorflow:global_step/sec: 654.884
INFO:tensorflow:loss = 95483510000.0, step = 63501 (0.153 sec)
INFO:tensorflow:global_step/sec: 664.623
INFO:tensorflow:loss = 123158200000.0, step = 63601 (0.151 sec)
INFO:tensorflow:global_step/sec: 719.752
INFO:tensorflow:loss = 24864158000.0, step = 63701 (0.139 sec)
INFO:tensorflow:global_step/sec: 703.303
INFO:tensorflow:loss = 46068440000.0, step = 63801 (0.141 sec)
INFO:tensorflow:global_step/sec: 667.094
INFO:tensorflow:loss =

INFO:tensorflow:global_step/sec: 709.869
INFO:tensorflow:loss = 41473745000.0, step = 70901 (0.141 sec)
INFO:tensorflow:global_step/sec: 663.658
INFO:tensorflow:loss = 72030626000.0, step = 71001 (0.147 sec)
INFO:tensorflow:global_step/sec: 676.444
INFO:tensorflow:loss = 14267996000.0, step = 71101 (0.148 sec)
INFO:tensorflow:global_step/sec: 706.479
INFO:tensorflow:loss = 44411040000.0, step = 71201 (0.143 sec)
INFO:tensorflow:global_step/sec: 639.051
INFO:tensorflow:loss = 49690485000.0, step = 71301 (0.156 sec)
INFO:tensorflow:global_step/sec: 660.48
INFO:tensorflow:loss = 38719840000.0, step = 71401 (0.153 sec)
INFO:tensorflow:global_step/sec: 741.902
INFO:tensorflow:loss = 36647375000.0, step = 71501 (0.134 sec)
INFO:tensorflow:global_step/sec: 782.575
INFO:tensorflow:loss = 65535300000.0, step = 71601 (0.130 sec)
INFO:tensorflow:global_step/sec: 585.783
INFO:tensorflow:loss = 20226580000.0, step = 71701 (0.168 sec)
INFO:tensorflow:global_step/sec: 668.739
INFO:tensorflow:loss = 1

INFO:tensorflow:global_step/sec: 619.87
INFO:tensorflow:loss = 41683505000.0, step = 78801 (0.158 sec)
INFO:tensorflow:global_step/sec: 714.042
INFO:tensorflow:loss = 66710454000.0, step = 78901 (0.140 sec)
INFO:tensorflow:global_step/sec: 704.229
INFO:tensorflow:loss = 81775420000.0, step = 79001 (0.143 sec)
INFO:tensorflow:global_step/sec: 757.491
INFO:tensorflow:loss = 96346290000.0, step = 79101 (0.132 sec)
INFO:tensorflow:global_step/sec: 768.758
INFO:tensorflow:loss = 63469322000.0, step = 79201 (0.131 sec)
INFO:tensorflow:global_step/sec: 693.621
INFO:tensorflow:loss = 33101283000.0, step = 79301 (0.144 sec)
INFO:tensorflow:global_step/sec: 710.384
INFO:tensorflow:loss = 142632240000.0, step = 79401 (0.140 sec)
INFO:tensorflow:global_step/sec: 748.48
INFO:tensorflow:loss = 29235374000.0, step = 79501 (0.132 sec)
INFO:tensorflow:global_step/sec: 744.696
INFO:tensorflow:loss = 15003158000.0, step = 79601 (0.135 sec)
INFO:tensorflow:global_step/sec: 738.231
INFO:tensorflow:loss = 5

INFO:tensorflow:global_step/sec: 725.531
INFO:tensorflow:loss = 36692230000.0, step = 86701 (0.134 sec)
INFO:tensorflow:global_step/sec: 661.704
INFO:tensorflow:loss = 88488800000.0, step = 86801 (0.155 sec)
INFO:tensorflow:global_step/sec: 651.055
INFO:tensorflow:loss = 31976784000.0, step = 86901 (0.151 sec)
INFO:tensorflow:global_step/sec: 693.818
INFO:tensorflow:loss = 25150610000.0, step = 87001 (0.143 sec)
INFO:tensorflow:global_step/sec: 732.488
INFO:tensorflow:loss = 42748367000.0, step = 87101 (0.140 sec)
INFO:tensorflow:global_step/sec: 722.324
INFO:tensorflow:loss = 83758560000.0, step = 87201 (0.135 sec)
INFO:tensorflow:global_step/sec: 753.12
INFO:tensorflow:loss = 99603870000.0, step = 87301 (0.134 sec)
INFO:tensorflow:global_step/sec: 666.649
INFO:tensorflow:loss = 168349750000.0, step = 87401 (0.149 sec)
INFO:tensorflow:global_step/sec: 692.981
INFO:tensorflow:loss = 34033766000.0, step = 87501 (0.147 sec)
INFO:tensorflow:global_step/sec: 657.726
INFO:tensorflow:loss = 

INFO:tensorflow:global_step/sec: 697.618
INFO:tensorflow:loss = 48097157000.0, step = 94601 (0.144 sec)
INFO:tensorflow:global_step/sec: 739.64
INFO:tensorflow:loss = 76456800000.0, step = 94701 (0.136 sec)
INFO:tensorflow:global_step/sec: 755.121
INFO:tensorflow:loss = 62492370000.0, step = 94801 (0.132 sec)
INFO:tensorflow:global_step/sec: 763.272
INFO:tensorflow:loss = 20584178000.0, step = 94901 (0.130 sec)
INFO:tensorflow:global_step/sec: 777.906
INFO:tensorflow:loss = 44031197000.0, step = 95001 (0.130 sec)
INFO:tensorflow:global_step/sec: 768.763
INFO:tensorflow:loss = 99242476000.0, step = 95101 (0.130 sec)
INFO:tensorflow:global_step/sec: 813.869
INFO:tensorflow:loss = 273322840000.0, step = 95201 (0.121 sec)
INFO:tensorflow:global_step/sec: 810.465
INFO:tensorflow:loss = 85033650000.0, step = 95301 (0.124 sec)
INFO:tensorflow:global_step/sec: 762.963
INFO:tensorflow:loss = 18570764000.0, step = 95401 (0.133 sec)
INFO:tensorflow:global_step/sec: 785.712
INFO:tensorflow:loss = 

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [21]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [22]:
pred_gen = model.predict(predict_input_func)

In [23]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/jf/g8tp5ct56hl6p6r66zl2knzc0000gn/T/tmptj6toi03/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [24]:
predictions

[{'predictions': array([294136.9], dtype=float32)},
 {'predictions': array([535417.4], dtype=float32)},
 {'predictions': array([170548.98], dtype=float32)},
 {'predictions': array([201139.55], dtype=float32)},
 {'predictions': array([465575.78], dtype=float32)},
 {'predictions': array([168132.92], dtype=float32)},
 {'predictions': array([201611.17], dtype=float32)},
 {'predictions': array([193157.38], dtype=float32)},
 {'predictions': array([271228.06], dtype=float32)},
 {'predictions': array([316422.8], dtype=float32)},
 {'predictions': array([188518.33], dtype=float32)},
 {'predictions': array([189681.75], dtype=float32)},
 {'predictions': array([173614.27], dtype=float32)},
 {'predictions': array([167246.36], dtype=float32)},
 {'predictions': array([293448.38], dtype=float32)},
 {'predictions': array([197531.36], dtype=float32)},
 {'predictions': array([168131.73], dtype=float32)},
 {'predictions': array([200270.05], dtype=float32)},
 {'predictions': array([180712.58], dtype=float32

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [25]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [26]:
from sklearn.metrics import mean_squared_error

In [27]:
mean_squared_error(y_test,final_preds)**0.5

80498.98409969876